In [7]:
import os
import collections
import numpy as np

from importlib import reload
from collections import Counter
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error



def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    
    dictionary = collections.Counter(all_words)
    list_to_remove = list(dictionary)
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(300) 
    return dictionary




def extract_features(mail_dir, dictionary): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),300))
    docID = 0;
 
    for fil in files:
        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i==2:
                    words=line.split()
                    for word in words:
                        wordID=0
                        for i,d in enumerate(dictionary):
                            if d[0]==word:
                                wordID=i
                                features_matrix[docID,wordID] = words.count(word)
                            
        docID = docID+1
    return features_matrix
        
        
        




# Create a dictionary of words with its frequency

train_dir = '/home/jovi/ling-spam/train-mails'

dictionary = make_Dictionary(train_dir)

# Prepare feature vectors per training mail and its labels

train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir, dictionary)


# Training SVM and Naive bayes classifier

model1 = MultinomialNB()
model2 = LinearSVC()
model1.fit(train_matrix,train_labels)
model2.fit(train_matrix,train_labels)

# Test the unseen mails for Spam
test_dir = '/home/jovi/ling-spam/test-mails'
test_matrix = extract_features(test_dir, dictionary)
test_labels = np.zeros(260)
test_labels[130:260] = 1
result1 = model1.predict(test_matrix)

result2 = model2.predict(test_matrix)
print('confusion matrix of Multinomial modal')
print(confusion_matrix(test_labels,result1))
print('error metrics of Multinomial modal')
print(np.sqrt(mean_squared_error(test_labels, result1)))
print('confusion matrix of linear SVC ')
print(confusion_matrix(test_labels,result2))
print('error metric of linear SVC ')
print(np.sqrt(mean_squared_error(test_labels, result2)))

confusion matrix of Multinomial modal
[[ 27 103]
 [ 41  89]]
error metrics of Multinomial modal
0.744208407535
confusion matrix of linear SVC 
[[62 68]
 [58 72]]
error metric of linear SVC 
0.696143221338
